In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load and preprocess the data
path = '/content/drive/MyDrive/NeurIPS/all_eye_images'
dataset_path = os.listdir(path)

im_size = 224
images = []
labels = []

for i in dataset_path:
    data_path = path + '/' + str(i)
    filenames = [i for i in os.listdir(data_path)]
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

images = np.array(images)
images = images.astype('float32')
images /= 255.0

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
labels = labels.reshape(-1, 1)

# One-hot encode labels
onehot_encoder = OneHotEncoder(sparse=False)
labels = onehot_encoder.fit_transform(labels)

# Shuffle and split data
images, labels = shuffle(images, labels, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, labels, test_size=0.2, random_state=415)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.15, random_state=415)

# Data augmentation
train_generator = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_generator = ImageDataGenerator(rescale = 1.0/255.)
test_generator = ImageDataGenerator()

base_model = VGG16(input_shape=(im_size, im_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')
])

model.compile(optimizer= RMSprop(learning_rate=0.0001, weight_decay=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Define callbacks
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(train_generator.flow(train_x, train_y, batch_size=32), epochs=13, callbacks=[callbacks], validation_data=val_generator.flow(val_x, val_y, batch_size=33))

# Evaluate the model
preds = model.evaluate(test_x, test_y)
print(f'Test Accuracy: {preds[1] * 100:.2f}%')

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


58889256/58889256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21137986 (80.64 MB)
Trainable params: 6423298 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
__________